In [ ]:
import pandas as pd
import numpy as np

# 1. Charger le dataset (adapte le chemin si nécessaire)
df = pd.read_csv("adult.csv")

# 2. Supprimer la colonne salaire binaire d'origine si elle existe
if 'income' in df.columns:
    df.drop(columns=['income'], inplace=True)

# 3. Fixer la graine aléatoire pour avoir les mêmes résultats à chaque exécution
np.random.seed(42)

# 4. Fonction pour générer un salaire cohérent et entier
def generate_salary(age, hours):
    base_hourly_rate = 7.5  # salaire horaire de base
    age_bonus = (age - 18) * 0.15  # +15 centimes par an d'âge au-delà de 18 ans
    hourly_rate = base_hourly_rate + age_bonus

    # Salaire brut hebdomadaire
    weekly_salary = hourly_rate * hours

    # Salaire annuel avec bruit
    noise = np.random.normal(0, weekly_salary * 0.05)  # bruit ±5% du salaire hebdo
    annual_salary = (weekly_salary * 52) + noise

    # Retourner un entier positif
    return int(max(annual_salary, 0))

# 5. Appliquer la fonction à chaque ligne du dataset
df['salary'] = df.apply(
    lambda row: generate_salary(row['age'], row['hours.per.week']), axis=1
)

# 6. Aperçu du résultat
print(df[['age', 'hours.per.week', 'salary']].head())

# 7. (Optionnel) Sauvegarder le dataset modifié
df.to_csv("adult_with_salaries.csv", index=False)




import pandas as pd
import numpy as np

# Charger le dataset depuis le fichier CSV
data = pd.read_csv("adult.csv")

# Remplacer les "?" (valeurs manquantes codées sous forme de texte) par de vrais NaN de NumPy
data = data.replace('?', np.nan)

# Vérifier combien de valeurs manquantes il y a par colonne
data.isnull().sum()

# Remplir les valeurs manquantes dans certaines colonnes avec la valeur la plus fréquente (mode)
for col in ['workclass', 'occupation', 'native.country']:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Vérifier le nombre de doublons dans le dataset
print(data.duplicated().sum())

# Supprimer toutes les lignes dupliquées pour ne garder qu'une occurrence unique
data = data.drop_duplicates()
print(data.shape)  # Afficher les dimensions du dataframe après suppression

# Supprimer les espaces inutiles au début et à la fin des chaînes de caractères
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Afficher les valeurs uniques pour chaque colonne de type "object" (catégorielle)
for col in data.select_dtypes(include='object').columns:
   print(col, data[col].unique())

# Définir une fonction pour détecter les valeurs aberrantes (outliers) dans une colonne numérique
def look_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # Premier quartile (25%)
    Q3 = df[column].quantile(0.75)  # Troisième quartile (75%)
    IQR = Q3 - Q1                   # Intervalle interquartile (IQR)
    lower = Q3 - IQR*1.5            # Limite inférieure
    upper = Q1 + IQR*1.5            # Limite supérieure
    # Retourner les lignes où la valeur est en dehors de ces bornes (potentiels outliers)
    return df[(df[column] < lower) & (df[column] > upper)]

# Afficher les 5 premières lignes pour vérifier l'état du dataframe
data.head()

# Vérifier les outliers dans certaines colonnes clés
columnc = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week', 'salary']
for col in columnc :
    da = look_outliers(data, col)
    print(col, da.shape)  # Afficher combien d'outliers ont été détectés par colonne

# Supprimer la colonne "fnlwgt" jugée peu utile pour l'analyse
data = data.drop(columns=['fnlwgt'])

# Sauvegarder le dataset nettoyé dans un nouveau fichier CSV
data.to_csv('adult_clean.csv', index=False)

# Charger les données nettoyées dans une base MySQL (table "adult_clean")
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://root:@localhost/datalab")
data.to_sql("adult_clean", con=engine, if_exists="replace", index=False)



   age  hours.per.week  salary
0   90              40   38082
1   82              18   16003
2   66              40   30595
3   54              40   26871
4   41              40   22770
(32561, 15)
0
(32561, 15)


C:\Users\JauresWilson\AppData\Local\Temp\ipykernel_13184\4287046020.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace=True)
C:\Users\JauresWilson\AppData\Local\Temp\ipykernel_13184\4287046020.py:64: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


32561

In [18]:
data.head()

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,salary
0,90,Private,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States,38082
1,82,Private,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,16003
2,66,Private,Some-college,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States,30595
3,54,Private,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,26871
4,41,Private,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,22770
